# General Dataset

In [1]:
# Cell 1: Setup and Imports for distilroberta Seed & Bootstrap Analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset, Dataset as HFDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
import json
import warnings
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import joblib
import random
from collections import Counter
import gc
from tqdm import tqdm
from scipy import stats

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create output directories
os.makedirs("./distilroberta_seed_analysis_results", exist_ok=True)
os.makedirs("./distilroberta_trained_models_seeds", exist_ok=True)

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print("✅ Libraries imported and setup complete!")

c:\Users\hankaixin\Desktop\multitask\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
✅ Libraries imported and setup complete!


In [2]:
# Cell 2: Utility Functions for Analysis
def set_random_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def print_memory_usage():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory - Allocated: {allocated:.2f} GB, Cached: {cached:.2f} GB")

print("Utility functions defined")

Utility functions defined


In [3]:
# Cell 3: distilroberta Model Architectures
class DistilrobertaSingleTaskTransformer(nn.Module):
    
    def __init__(
        self,
        model_name: str = "distilroberta-base",
        num_classes: int = 3,
        hidden_dropout_prob: float = 0.1,
        attention_dropout_prob: float = 0.1,
        classifier_dropout: float = 0.1
    ):
        super().__init__()
        self.num_classes = num_classes
        
        # Load distilroberta model
        config = AutoConfig.from_pretrained(model_name)
        config.hidden_dropout_prob = hidden_dropout_prob
        config.attention_probs_dropout_prob = attention_dropout_prob
        
        self.distilroberta = AutoModel.from_pretrained(model_name, config=config)
        
        # Classification head
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(self.distilroberta.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        # Get distilroberta outputs
        outputs = self.distilroberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token
        pooled_output = self.dropout(pooled_output)
        
        # Classification
        logits = self.classifier(pooled_output)
        
        return {'logits': logits}

class DistilrobertaMultiTaskTransformer(nn.Module):
    
    def __init__(
        self,
        model_name: str = "distilroberta-base",
        sentiment_num_classes: int = 3,
        emotion_num_classes: int = 6,
        hidden_dropout_prob: float = 0.1,
        attention_dropout_prob: float = 0.1,
        classifier_dropout: float = 0.1
    ):
        super().__init__()
        self.sentiment_num_classes = sentiment_num_classes
        self.emotion_num_classes = emotion_num_classes
        
        # Load BERTwe   et model
        config = AutoConfig.from_pretrained(model_name)
        config.hidden_dropout_prob = hidden_dropout_prob
        config.attention_probs_dropout_prob = attention_dropout_prob
        
        self.distilroberta = AutoModel.from_pretrained(model_name, config=config)
        
        hidden_size = self.distilroberta.config.hidden_size
        
        # Task-specific attention layers
        self.sentiment_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        self.emotion_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        # Shared attention for common features
        self.shared_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        # Layer normalization
        self.sentiment_norm = nn.LayerNorm(hidden_size)
        self.emotion_norm = nn.LayerNorm(hidden_size)
        self.shared_norm = nn.LayerNorm(hidden_size)
        
        # Dropout layers
        self.sentiment_dropout = nn.Dropout(classifier_dropout)
        self.emotion_dropout = nn.Dropout(classifier_dropout)
        self.shared_dropout = nn.Dropout(classifier_dropout)
        
        # Classification heads
        self.sentiment_classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(hidden_size, sentiment_num_classes)
        )
        
        self.emotion_classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(hidden_size, emotion_num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for module in [self.sentiment_classifier, self.emotion_classifier]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.zeros_(layer.bias)
    
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Dict[str, torch.Tensor]:
        # Shared encoder
        encoder_outputs = self.distilroberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        sequence_output = encoder_outputs.last_hidden_state
        
        # Apply shared attention
        shared_attended, _ = self.shared_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        shared_attended = self.shared_norm(shared_attended + sequence_output)
        shared_attended = self.shared_dropout(shared_attended)
        shared_pooled = shared_attended[:, 0, :]
        
        outputs = {}
        
        # Sentiment branch
        sentiment_attended, _ = self.sentiment_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        sentiment_attended = self.sentiment_norm(sentiment_attended + sequence_output)
        sentiment_attended = self.sentiment_dropout(sentiment_attended)
        sentiment_pooled = sentiment_attended[:, 0, :]
        sentiment_features = torch.cat([shared_pooled, sentiment_pooled], dim=-1)
        sentiment_logits = self.sentiment_classifier(sentiment_features)
        outputs["sentiment_logits"] = sentiment_logits
        
        # Emotion branch
        emotion_attended, _ = self.emotion_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        emotion_attended = self.emotion_norm(emotion_attended + sequence_output)
        emotion_attended = self.emotion_dropout(emotion_attended)
        emotion_pooled = emotion_attended[:, 0, :]
        emotion_features = torch.cat([shared_pooled, emotion_pooled], dim=-1)
        emotion_logits = self.emotion_classifier(emotion_features)
        outputs["emotion_logits"] = emotion_logits
        
        return outputs

print("distilroberta model architectures defined")

distilroberta model architectures defined


In [4]:
# Cell 4: Dataset Classes for distilroberta
class DistilrobertaDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class DistilrobertaMultiTaskDataset(Dataset):
    def __init__(self, texts: List[str], sentiment_labels: List[int], 
                 emotion_labels: List[int], tokenizer, max_length: int = 128):
        self.texts = texts
        self.sentiment_labels = sentiment_labels
        self.emotion_labels = emotion_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        sentiment_label = self.sentiment_labels[idx]
        emotion_label = self.emotion_labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sentiment_labels': torch.tensor(sentiment_label, dtype=torch.long),
            'emotion_labels': torch.tensor(emotion_label, dtype=torch.long)
        }

print("distilroberta dataset classes defined")

distilroberta dataset classes defined


In [5]:
# Cell 5: Modified Data Loading Functions for General Dataset Evaluation
from typing import Tuple, Dict

def load_external_datasets() -> Tuple[Dict, Dict]:
    print("Loading external datasets...")
    
    # Load SST-2 for sentiment
    try:
        sst2_dataset = load_dataset("sst2")
        sentiment_data = {
            'train': sst2_dataset['train'],
            'validation': sst2_dataset['validation']
        }
        print(f"SST-2 dataset loaded: {len(sentiment_data['train'])} train, {len(sentiment_data['validation'])} validation samples")
    except Exception as e:
        print(f"Could not load SST-2: {e}")
        raise
    
    # Load GoEmotions for emotion
    try:
        emotions_dataset = load_dataset("go_emotions", "simplified")
        emotion_data = {
            'train': emotions_dataset['train'],
            'validation': emotions_dataset['validation'],
            'test': emotions_dataset['test']  # GoEmotions has a test split
        }
        print(f"GoEmotions dataset loaded: {len(emotion_data['train'])} train, {len(emotion_data['validation'])} validation, {len(emotion_data['test'])} test samples")
    except Exception as e:
        print(f"Could not load GoEmotions: {e}")
        raise
    
    return sentiment_data, emotion_data

def prepare_sst2_evaluation_data(sentiment_data: Dict, max_samples: int = 1000) -> Dict:
    print("Preparing SST-2 evaluation data...")
    
    # Use validation split for evaluation
    eval_texts = sentiment_data['validation']['sentence'][:max_samples]
    eval_labels_raw = sentiment_data['validation']['label'][:max_samples]
    
    # Convert SST-2 binary to 3-class sentiment (same as training)
    eval_labels = []
    for label in eval_labels_raw:
        if label == 0:  # Negative
            eval_labels.append(0)
        elif label == 1:  # Positive
            if np.random.random() < 0.15:  # 15% chance to be neutral (same as training)
                eval_labels.append(1)  # Neutral
            else:
                eval_labels.append(2)  # Positive
    
    # Create encoder that matches training
    sentiment_encoder = LabelEncoder()
    sentiment_encoder.classes_ = np.array(['Negative', 'Neutral', 'Positive'])
    
    sst2_eval_data = {
        'texts': eval_texts,
        'sentiment_labels': eval_labels,
        'sentiment_encoder': sentiment_encoder
    }
    
    print(f"✅ SST-2 evaluation data prepared: {len(sst2_eval_data['texts'])} samples")
    print(f"   Sentiment classes: {list(sentiment_encoder.classes_)}")
    
    return sst2_eval_data

def prepare_goemotions_evaluation_data(emotion_data: Dict, max_samples: int = 1000) -> Dict:
    print("Preparing GoEmotions evaluation data...")
    
    # Use test split for evaluation (or validation if test not available)
    eval_split = emotion_data.get('test', emotion_data.get('validation'))
    eval_texts_all = eval_split['text']
    eval_labels_all = eval_split['labels']
    
    eval_texts = []
    eval_labels = []
    count = 0
    
    for i, label in enumerate(eval_labels_all):
        if count >= max_samples:
            break
        if isinstance(label, list):
            if label and label[0] in range(6):
                eval_texts.append(eval_texts_all[i])
                eval_labels.append(label[0])
                count += 1
        else:
            if label in range(6):
                eval_texts.append(eval_texts_all[i])
                eval_labels.append(label)
                count += 1
    
    # Create encoder that matches training
    emotion_encoder = LabelEncoder()
    emotion_encoder.classes_ = np.array(['Anger', 'Fear', 'Joy', 'No Emotion', 'Sadness', 'Surprise'])
    
    goemotions_eval_data = {
        'texts': eval_texts,
        'emotion_labels': eval_labels,
        'emotion_encoder': emotion_encoder
    }
    
    print(f"✅ GoEmotions evaluation data prepared: {len(goemotions_eval_data['texts'])} samples")
    print(f"   Emotion classes: {list(emotion_encoder.classes_)}")
    
    return goemotions_eval_data

def prepare_multitask_evaluation_data(sst2_eval_data: Dict, goemotions_eval_data: Dict) -> Dict:
    print("Preparing multitask evaluation data...")
    
    # Take minimum length to ensure both tasks have same number of samples
    min_length = min(len(sst2_eval_data['texts']), len(goemotions_eval_data['texts']))
    
    multitask_eval_data = {
        'texts': sst2_eval_data['texts'][:min_length],
        'sentiment_labels': sst2_eval_data['sentiment_labels'][:min_length],
        'emotion_labels': goemotions_eval_data['emotion_labels'][:min_length],
        'sentiment_encoder': sst2_eval_data['sentiment_encoder'],
        'emotion_encoder': goemotions_eval_data['emotion_encoder']
    }
    
    print(f"✅ Multitask evaluation data prepared: {len(multitask_eval_data['texts'])} samples")
    
    return multitask_eval_data

def prepare_distilroberta_training_data(sentiment_data: Dict, emotion_data: Dict, max_samples: int = 3000) -> Dict:
    print("Preparing distilroberta training data...")
    
    # Process sentiment data (SST-2)
    sentiment_texts = sentiment_data['train']['sentence'][:max_samples]
    sentiment_labels = sentiment_data['train']['label'][:max_samples]
    
    # Process emotion data (filter to first 6 classes)
    emotion_texts = []
    emotion_labels = []
    count = 0
    
    for i, label in enumerate(emotion_data['train']['labels']):
        if count >= max_samples:
            break
        if isinstance(label, list):
            if label and label[0] in range(6):  # Only use first 6 emotions
                emotion_texts.append(emotion_data['train']['text'][i])
                emotion_labels.append(label[0])
                count += 1
        else:
            if label in range(6):
                emotion_texts.append(emotion_data['train']['text'][i])
                emotion_labels.append(label)
                count += 1
    
    # Create encoders
    sentiment_encoder = LabelEncoder()
    emotion_encoder = LabelEncoder()
    
    # For SST-2: 0 = Negative, 1 = Positive
    sentiment_encoder.classes_ = np.array(['Negative', 'Positive'])
    
    # For GoEmotions: First 6 emotions
    emotion_encoder.classes_ = np.array(['Anger', 'Fear', 'Joy', 'No Emotion', 'Sadness', 'Surprise'])
    
    training_data = {
        'sentiment_data': {
            'texts': sentiment_texts,
            'labels': sentiment_labels,
            'encoder': sentiment_encoder
        },
        'emotion_data': {
            'texts': emotion_texts,
            'labels': emotion_labels,
            'encoder': emotion_encoder
        }
    }
    
    print(f"✅ Training data prepared:")
    print(f"   Sentiment: {len(sentiment_texts)} samples")
    print(f"   Sentiment classes: {list(sentiment_encoder.classes_)}")
    print(f"   Emotion: {len(emotion_texts)} samples")
    print(f"   Emotion classes: {list(emotion_encoder.classes_)}")
    
    return training_data

print("✅ Modified data loading functions defined!")

✅ Modified data loading functions defined!


In [6]:
# Cell 6: distilroberta Training Functions with Best Parameters
def train_distilroberta_single_task(
    task_type: str,  # 'sentiment' or 'emotion'
    best_params: Dict,
    seed: int,
    training_data: Dict,
    max_samples: int = 5000
) -> Tuple[any, LabelEncoder]:
    
    print(f"🚀 Training distilroberta {task_type} model with seed {seed}")
    set_random_seed(seed)
    clear_memory()
    
    # Get appropriate data
    if task_type == 'sentiment':
        texts = training_data['sentiment_data']['texts'][:max_samples]
        labels = training_data['sentiment_data']['labels'][:max_samples]
        encoder = training_data['sentiment_data']['encoder']
        num_classes = 3
    else:  # emotion
        texts = training_data['emotion_data']['texts'][:max_samples]
        labels = training_data['emotion_data']['labels'][:max_samples]
        encoder = training_data['emotion_data']['encoder']
        num_classes = 6
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model
    model = DistilrobertaSingleTaskTransformer(
        model_name='distilroberta-base',
        num_classes=num_classes,
        hidden_dropout_prob=best_params['hidden_dropout_prob'],
        attention_dropout_prob=best_params['hidden_dropout_prob'],
        classifier_dropout=best_params['classifier_dropout']
    ).to(device)
    
    # Create dataset and dataloader
    dataset = DistilrobertaDataset(texts, labels, tokenizer, max_length=128)
    dataloader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=True)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
    
    total_steps = len(dataloader) * 3  # 3 epochs
    warmup_steps = int(total_steps * best_params['warmup_ratio'])
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    model.train()
    print(f"Starting training for 3 epochs...")
    
    for epoch in range(3):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_batch = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs['logits'], labels_batch)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/3, Average Loss: {avg_loss:.4f}")
    
    # Save model
    output_dir = f"./distilroberta_trained_models_seeds/distilroberta_{task_type}_seed_{seed}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model state dict
    torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
    
    # Save config
    config = {
        "model_name": "distilroberta-base",
        "num_classes": num_classes,
        "task_type": task_type,
        "model_type": "DistilrobertaSingleTaskTransformer"
    }
    with open(os.path.join(output_dir, "config.json"), 'w') as f:
        json.dump(config, f, indent=2)
    
    # Save tokenizer and encoder
    tokenizer.save_pretrained(output_dir)
    joblib.dump(encoder, os.path.join(output_dir, f'{task_type}_encoder.pkl'))
    
    print(f"✅ distilroberta {task_type} model trained and saved with seed {seed}")
    clear_memory()
    
    return model, encoder

def train_distilroberta_multitask(
    best_params: Dict,
    seed: int,
    training_data: Dict,
    max_samples: int = 2000
) -> Tuple[any, LabelEncoder, LabelEncoder]:
    
    print(f"🚀 Training distilroberta multitask model with seed {seed}")
    set_random_seed(seed)
    clear_memory()
    
    # Prepare multitask data (combine sentiment and emotion data)
    min_length = min(len(training_data['sentiment_data']['texts']), 
                     len(training_data['emotion_data']['texts']))
    min_length = min(min_length, max_samples)
    
    combined_texts = training_data['sentiment_data']['texts'][:min_length]
    combined_sentiment_labels = training_data['sentiment_data']['labels'][:min_length]
    combined_emotion_labels = training_data['emotion_data']['labels'][:min_length]
    
    sentiment_encoder = training_data['sentiment_data']['encoder']
    emotion_encoder = training_data['emotion_data']['encoder']
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model
    model = DistilrobertaMultiTaskTransformer(
        model_name='distilroberta-base',
        sentiment_num_classes=3,
        emotion_num_classes=6,
        hidden_dropout_prob=best_params['hidden_dropout_prob'],
        attention_dropout_prob=best_params['hidden_dropout_prob'],
        classifier_dropout=best_params['classifier_dropout']
    ).to(device)
    
    # Create dataset and dataloader
    dataset = DistilrobertaMultiTaskDataset(
        combined_texts, combined_sentiment_labels, combined_emotion_labels, 
        tokenizer, max_length=128
    )
    dataloader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=True)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
    
    total_steps = len(dataloader) * 3  # 3 epochs
    warmup_steps = int(total_steps * best_params['warmup_ratio'])
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # Loss functions
    sentiment_criterion = nn.CrossEntropyLoss()
    emotion_criterion = nn.CrossEntropyLoss()
    
    alpha = best_params['alpha']
    
    # Training loop
    model.train()
    print(f"Starting training for 3 epochs...")
    
    for epoch in range(3):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            sentiment_labels = batch['sentiment_labels'].to(device)
            emotion_labels = batch['emotion_labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Calculate losses
            sentiment_loss = sentiment_criterion(outputs['sentiment_logits'], sentiment_labels)
            emotion_loss = emotion_criterion(outputs['emotion_logits'], emotion_labels)
            
            # Combined loss
            total_loss_batch = alpha * sentiment_loss + (1 - alpha) * emotion_loss
            total_loss += total_loss_batch.item()
            
            # Backward pass
            total_loss_batch.backward()
            optimizer.step()
            scheduler.step()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/3, Average Loss: {avg_loss:.4f}")
    
    # Save model
    output_dir = f"./distilroberta_trained_models_seeds/distilroberta_multitask_seed_{seed}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model state dict
    torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
    
    # Save config
    config = {
        "model_name": "distilroberta-base",
        "sentiment_num_classes": 3,
        "emotion_num_classes": 6,
        "model_type": "DistilrobertaMultiTaskTransformer"
    }
    with open(os.path.join(output_dir, "config.json"), 'w') as f:
        json.dump(config, f, indent=2)
    
    # Save tokenizer and encoders
    tokenizer.save_pretrained(output_dir)
    joblib.dump(sentiment_encoder, os.path.join(output_dir, 'sentiment_encoder.pkl'))
    joblib.dump(emotion_encoder, os.path.join(output_dir, 'emotion_encoder.pkl'))
    
    print(f"distilroberta multitask model trained and saved with seed {seed}")
    clear_memory()
    
    return model, sentiment_encoder, emotion_encoder

print("distilroberta training functions defined")

distilroberta training functions defined


In [7]:
# Cell 7: Evaluation Functions for distilroberta Models
def evaluate_distilroberta_single_task(model, tokenizer, label_encoder, reddit_data: Dict, task_type: str) -> Dict:
    
    model.eval()
    model.to(device)
    
    texts = reddit_data['texts']
    true_labels = reddit_data[f'{task_type}_labels']
    
    predictions = []
    confidences = []
    
    with torch.no_grad():
        for i in range(0, len(texts), 16):  # Batch size 16
            batch_texts = texts[i:i+16]
            
            # Tokenize
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=128
            )
            
            inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
            
            # Forward pass
            outputs = model(**inputs)
            logits = outputs['logits']
            probs = F.softmax(logits, dim=-1)
            preds = torch.argmax(logits, dim=-1)
            
            # Collect results
            for j in range(len(batch_texts)):
                pred_id = preds[j].item()
                confidence = probs[j][pred_id].item()
                
                # Handle out of range predictions
                if pred_id >= len(label_encoder.classes_):
                    pred_id = 0
                
                predictions.append(pred_id)
                confidences.append(confidence)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    macro_f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'predictions': predictions,
        'confidences': confidences,
        'true_labels': true_labels
    }

def evaluate_distilroberta_multitask(model, tokenizer, sentiment_encoder, emotion_encoder, 
                               reddit_data: Dict, max_length: int = 128) -> Dict:
    
    model.eval()
    model.to(device)
    
    texts = reddit_data['texts']
    true_sentiment_labels = reddit_data['sentiment_labels']
    true_emotion_labels = reddit_data['emotion_labels']
    
    sentiment_predictions = []
    emotion_predictions = []
    sentiment_confidences = []
    emotion_confidences = []
    
    with torch.no_grad():
        for i in range(0, len(texts), 8):  # Smaller batch size for multitask
            batch_texts = texts[i:i+8]
            
            # Tokenize
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=max_length
            )
            
            inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
            
            # Forward pass
            outputs = model(**inputs)
            
            # Process sentiment
            sentiment_logits = outputs['sentiment_logits']
            sentiment_probs = F.softmax(sentiment_logits, dim=-1)
            sentiment_preds = torch.argmax(sentiment_logits, dim=-1)
            
            # Process emotion
            emotion_logits = outputs['emotion_logits']
            emotion_probs = F.softmax(emotion_logits, dim=-1)
            emotion_preds = torch.argmax(emotion_logits, dim=-1)
            
            # Collect results
            for j in range(len(batch_texts)):
                # Sentiment
                sent_id = sentiment_preds[j].item()
                sent_conf = sentiment_probs[j][sent_id].item()
                if sent_id >= len(sentiment_encoder.classes_):
                    sent_id = 0
                sentiment_predictions.append(sent_id)
                sentiment_confidences.append(sent_conf)
                
                # Emotion
                emot_id = emotion_preds[j].item()
                emot_conf = emotion_probs[j][emot_id].item()
                if emot_id >= len(emotion_encoder.classes_):
                    emot_id = 0
                emotion_predictions.append(emot_id)
                emotion_confidences.append(emot_conf)
    
    # Calculate metrics
    sentiment_accuracy = accuracy_score(true_sentiment_labels, sentiment_predictions)
    sentiment_f1 = f1_score(true_sentiment_labels, sentiment_predictions, average='macro', zero_division=0)
    
    emotion_accuracy = accuracy_score(true_emotion_labels, emotion_predictions)
    emotion_f1 = f1_score(true_emotion_labels, emotion_predictions, average='macro', zero_division=0)
    
    return {
        'sentiment': {
            'accuracy': sentiment_accuracy,
            'macro_f1': sentiment_f1,
            'predictions': sentiment_predictions,
            'confidences': sentiment_confidences
        },
        'emotion': {
            'accuracy': emotion_accuracy,
            'macro_f1': emotion_f1,
            'predictions': emotion_predictions,
            'confidences': emotion_confidences
        },
        'combined_accuracy': (sentiment_accuracy + emotion_accuracy) / 2,
        'combined_f1': (sentiment_f1 + emotion_f1) / 2
    }

print("distilroberta evaluation functions defined")

distilroberta evaluation functions defined


In [8]:
# Cell 8: Modified distilroberta Random Seed Analysis Function
def run_distilroberta_seed_analysis(
    seeds: List[int] = [42, 123, 456, 789, 999],
    max_training_samples: int = 3000,
    max_eval_samples: int = 1000
):
    
    print("🎲 STARTING distilroberta RANDOM SEED ANALYSIS")
    print("=" * 70)
    print(f"Seeds to test: {seeds}")
    print(f"Max training samples per dataset: {max_training_samples}")
    print(f"Max evaluation samples per dataset: {max_eval_samples}")
    
    # Load external datasets
    print("\n📂 Loading external datasets...")
    sentiment_data, emotion_data = load_external_datasets()
    
    # Prepare training data
    print("\n🔄 Preparing distilroberta training data...")
    training_data = prepare_distilroberta_training_data(sentiment_data, emotion_data, max_training_samples)
    
    # Prepare evaluation data
    print("\n📂 Preparing evaluation datasets...")
    sst2_eval_data = prepare_sst2_evaluation_data(sentiment_data, max_eval_samples)
    goemotions_eval_data = prepare_goemotions_evaluation_data(emotion_data, max_eval_samples)
    multitask_eval_data = prepare_multitask_evaluation_data(sst2_eval_data, goemotions_eval_data)
    
    # Define best parameters for each distilroberta model
    best_params = {
        'sentiment': {
            'learning_rate': 5.262490902114904e-05,
            'batch_size': 16,
            'num_epochs': 6,
            'warmup_ratio': 0.18324426408004219,
            'weight_decay': 0.029110519961044856,
            'hidden_dropout_prob': 0.1363649934414201,
            'classifier_dropout': 0.13668090197068677
        },
        'emotion': {
            'learning_rate': 4.166863122305896e-05,
            'batch_size': 16,
            'num_epochs': 5,
            'warmup_ratio': 0.15924145688620425,
            'weight_decay': 0.014180537144799797,
            'hidden_dropout_prob': 0.22150897038028766,
            'classifier_dropout': 0.1341048247374583
        },
        'multitask': {
            'learning_rate': 6.251028636335231e-05,
            'batch_size': 32,
            'warmup_ratio': 0.12123391106782762,
            'weight_decay': 0.02636424704863906,
            'hidden_dropout_prob': 0.13668090197068677,
            'classifier_dropout': 0.16084844859190756,
            'alpha': 0.5049512863264476
        }
    }
  
    # Store results for each seed
    all_results = {}
    
    for seed in seeds:
        print(f"\n🌱 TRAINING AND EVALUATING distilroberta WITH SEED {seed}")
        print("-" * 60)
        
        seed_results = {}
        
        # 1. Train and evaluate distilroberta Sentiment on SST-2
        print(f"\n1️⃣ distilroberta Sentiment on SST-2 (Seed {seed})")
        model, encoder = train_distilroberta_single_task(
            'sentiment', best_params['sentiment'], seed, 
            training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_{seed}")
        
        # Evaluate on SST-2 validation set
        results = evaluate_distilroberta_single_task(model, tokenizer, encoder, sst2_eval_data, 'sentiment')
        seed_results['distilroberta_sentiment'] = results
        print(f"   Accuracy: {results['accuracy']:.4f}, Macro F1: {results['macro_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        # 2. Train and evaluate distilroberta Emotion on GoEmotions
        print(f"\n2️⃣ distilroberta Emotion on GoEmotions (Seed {seed})")
        model, encoder = train_distilroberta_single_task(
            'emotion', best_params['emotion'], seed,
            training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_emotion_seed_{seed}")
        
        # Evaluate on GoEmotions test set
        results = evaluate_distilroberta_single_task(model, tokenizer, encoder, goemotions_eval_data, 'emotion')
        seed_results['distilroberta_emotion'] = results
        print(f"   Accuracy: {results['accuracy']:.4f}, Macro F1: {results['macro_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        # 3. Train and evaluate distilroberta Multitask on both datasets
        print(f"\n3️⃣ distilroberta Multitask on SST-2 + GoEmotions (Seed {seed})")
        model, sent_enc, emot_enc = train_distilroberta_multitask(
            best_params['multitask'], seed, training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_multitask_seed_{seed}")
        
        # Evaluate on combined test sets
        results = evaluate_distilroberta_multitask(
            model, tokenizer, sent_enc, emot_enc, multitask_eval_data, 128
        )
        seed_results['distilroberta_multitask'] = results
        print(f"   Sentiment - Accuracy: {results['sentiment']['accuracy']:.4f}, F1: {results['sentiment']['macro_f1']:.4f}")
        print(f"   Emotion - Accuracy: {results['emotion']['accuracy']:.4f}, F1: {results['emotion']['macro_f1']:.4f}")
        print(f"   Combined - Accuracy: {results['combined_accuracy']:.4f}, F1: {results['combined_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        all_results[seed] = seed_results
        
        print(f"\n✅ Completed evaluation for seed {seed}")
    
    # Analyze stability across seeds
    print(f"\n📊 ANALYZING distilroberta STABILITY ACROSS SEEDS")
    print("=" * 70)
    
    stability_analysis = analyze_distilroberta_seed_stability(all_results, seeds)
    
    # Save results
    save_distilroberta_results(all_results, stability_analysis, seeds)
    
    return all_results, stability_analysis

print("✅ Modified distilroberta random seed analysis function defined!")

✅ Modified distilroberta random seed analysis function defined!


In [9]:
# Cell 9: distilroberta Stability Analysis Functions
def analyze_distilroberta_seed_stability(all_results: Dict, seeds: List[int]) -> Dict:
    
    stability_stats = {}
    
    # Define model-task combinations
    evaluations = [
        ('distilroberta_sentiment', 'sentiment'),
        ('distilroberta_emotion', 'emotion'),
        ('distilroberta_multitask', 'sentiment'),
        ('distilroberta_multitask', 'emotion')
    ]
    
    for model_name, task in evaluations:
        print(f"\n🔍 {model_name.upper()} - {task.upper()}")
        
        accuracies = []
        f1_scores = []
        
        for seed in seeds:
            if model_name in all_results[seed]:
                result = all_results[seed][model_name]
                
                if model_name.endswith('_multitask'):
                    acc = result[task]['accuracy']
                    f1 = result[task]['macro_f1']
                else:
                    acc = result['accuracy']
                    f1 = result['macro_f1']
                
                accuracies.append(acc)
                f1_scores.append(f1)
        
        if accuracies:
            acc_mean = np.mean(accuracies)
            acc_std = np.std(accuracies)
            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            
            stability_stats[f"{model_name}_{task}"] = {
                'accuracy_mean': acc_mean,
                'accuracy_std': acc_std,
                'f1_mean': f1_mean,
                'f1_std': f1_std,
                'accuracy_values': accuracies,
                'f1_values': f1_scores
            }
            
            print(f"   Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
            print(f"   Macro F1: {f1_mean:.4f} ± {f1_std:.4f}")
    
    return stability_stats

def save_distilroberta_results(all_results: Dict, stability_analysis: Dict, seeds: List[int]):
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save raw results
    results_file = f"./distilroberta_seed_analysis_results/distilroberta_raw_results_{timestamp}.json"
    with open(results_file, 'w') as f:
        # Convert numpy types to Python types for JSON serialization
        serializable_results = {}
        for seed, seed_results in all_results.items():
            serializable_results[str(seed)] = {}
            for model, results in seed_results.items():
                if isinstance(results, dict):
                    serializable_results[str(seed)][model] = {}
                    for key, value in results.items():
                        if isinstance(value, dict):
                            serializable_results[str(seed)][model][key] = {
                                k: float(v) if isinstance(v, (np.floating, np.integer)) else 
                                   [float(x) if isinstance(x, (np.floating, np.integer)) else x for x in v] if isinstance(v, list) else v
                                for k, v in value.items()
                            }
                        else:
                            serializable_results[str(seed)][model][key] = float(value) if isinstance(value, (np.floating, np.integer)) else value
        
        json.dump(serializable_results, f, indent=2)
    
    # Save stability analysis
    stability_file = f"./distilroberta_seed_analysis_results/distilroberta_stability_analysis_{timestamp}.json"
    with open(stability_file, 'w') as f:
        serializable_stability = {}
        for key, stats in stability_analysis.items():
            serializable_stability[key] = {
                k: float(v) if isinstance(v, (np.floating, np.integer)) else 
                   [float(x) for x in v] if isinstance(v, list) else v
                for k, v in stats.items()
            }
        json.dump(serializable_stability, f, indent=2)
    
    # Create summary report
    summary_file = f"./distilroberta_seed_analysis_results/distilroberta_summary_report_{timestamp}.txt"
    with open(summary_file, 'w') as f:
        f.write("distilroberta RANDOM SEED ANALYSIS SUMMARY REPORT\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Seeds tested: {seeds}\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("STABILITY ANALYSIS (Mean ± Std)\n")
        f.write("-" * 40 + "\n")
        
        for key, stats in stability_analysis.items():
            model_task = key.replace('_', ' ').title()
            f.write(f"\n{model_task}:\n")
            f.write(f"  Accuracy: {stats['accuracy_mean']:.4f} ± {stats['accuracy_std']:.4f}\n")
            f.write(f"  Macro F1: {stats['f1_mean']:.4f} ± {stats['f1_std']:.4f}\n")
        
        f.write(f"\nBest Performers (by mean F1 score):\n")
        f.write("-" * 30 + "\n")
        
        # Find best performers
        sentiment_best = max([k for k in stability_analysis.keys() if 'sentiment' in k], 
                           key=lambda x: stability_analysis[x]['f1_mean'])
        emotion_best = max([k for k in stability_analysis.keys() if 'emotion' in k], 
                         key=lambda x: stability_analysis[x]['f1_mean'])
        
        f.write(f"Sentiment: {sentiment_best.replace('_', ' ').title()} ")
        f.write(f"(F1: {stability_analysis[sentiment_best]['f1_mean']:.4f})\n")
        f.write(f"Emotion: {emotion_best.replace('_', ' ').title()} ")
        f.write(f"(F1: {stability_analysis[emotion_best]['f1_mean']:.4f})\n")
    
    print(f"\n💾 distilroberta results saved:")
    print(f"   Raw results: {results_file}")
    print(f"   Stability analysis: {stability_file}")
    print(f"   Summary report: {summary_file}")

print("train_distilroberta_multitask stability analysis functions defined")

train_distilroberta_multitask stability analysis functions defined


In [26]:
# Cell 10: Run distilroberta Random Seed Analysis (Fixed)

# Clear any previous results
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Run distilroberta random seed analysis with the modified function signature
try:
    all_results, stability_analysis = run_distilroberta_seed_analysis(
        seeds=[42, 123, 456, 789, 999],  # 5 different seeds
        max_training_samples=3000,  # Reduced for faster training
        max_eval_samples=1000  # Max evaluation samples per dataset
    )
    
    print("\nDISTILROBERTA RANDOM SEED ANALYSIS COMPLETED!")
    print("=" * 60)
    print("Check the './distilroberta_seed_analysis_results/' directory for detailed results.")
    
    # Display quick summary
    print("\n📊 QUICK STABILITY SUMMARY:")
    print("-" * 40)
    
    # Updated to match the actual structure of stability_analysis
    for key, stats in stability_analysis.items():
        model_task = key.replace('_', ' ').title()
        print(f"\n{model_task}:")
        print(f"  Accuracy: {stats['accuracy_mean']:.3f} ± {stats['accuracy_std']:.3f}")
        print(f"  F1 Score: {stats['f1_mean']:.3f} ± {stats['f1_std']:.3f}")

except Exception as e:
    print(f"❌ Error during analysis: {str(e)}")
    print("🔧 Try restarting the kernel and running cells 1-9 again.")

🎲 STARTING distilroberta RANDOM SEED ANALYSIS
Seeds to test: [42, 123, 456, 789, 999]
Max training samples per dataset: 3000
Max evaluation samples per dataset: 1000

📂 Loading external datasets...
Loading external datasets...
SST-2 dataset loaded: 67349 train, 872 validation samples
GoEmotions dataset loaded: 43410 train, 5426 validation, 5427 test samples

🔄 Preparing distilroberta training data...
Preparing distilroberta training data...
✅ Training data prepared:
   Sentiment: 3000 samples
   Sentiment classes: [np.str_('Negative'), np.str_('Positive')]
   Emotion: 3000 samples
   Emotion classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]

📂 Preparing evaluation datasets...
Preparing SST-2 evaluation data...
✅ SST-2 evaluation data prepared: 872 samples
   Sentiment classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
Preparing GoEmotions evaluation data...
✅ GoEmotions evaluation data prepa

In [10]:
# Cell 11: distilroberta Bootstrap Analysis Functions
def load_distilroberta_model_for_bootstrap(model_path: str, model_type: str):
    print(f"📥 Loading distilroberta {model_type} model from {model_path}...")
    
    # Load config
    with open(os.path.join(model_path, 'config.json'), 'r') as f:
        config = json.load(f)
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    if model_type == "multitask":
        # Load multitask model
        model = DistilrobertaMultiTaskTransformer(
            model_name="distilroberta-base",
            sentiment_num_classes=config['sentiment_num_classes'],
            emotion_num_classes=config['emotion_num_classes']
        )
        
        # Load weights
        state_dict = torch.load(os.path.join(model_path, 'pytorch_model.bin'), map_location=device)
        model.load_state_dict(state_dict)
        model.to(device)
        
        # Load encoders
        sentiment_encoder = joblib.load(os.path.join(model_path, 'sentiment_encoder.pkl'))
        emotion_encoder = joblib.load(os.path.join(model_path, 'emotion_encoder.pkl'))
        
        return model, tokenizer, sentiment_encoder, emotion_encoder
        
    else:
        # Load single-task model
        model = DistilrobertaSingleTaskTransformer(
            model_name="distilroberta-base",
            num_classes=config['num_classes']
        )
        
        # Load weights
        state_dict = torch.load(os.path.join(model_path, 'pytorch_model.bin'), map_location=device)
        model.load_state_dict(state_dict)
        model.to(device)
        
        # Load encoder
        encoder = joblib.load(os.path.join(model_path, f'{config["task_type"]}_encoder.pkl'))
        
        return model, tokenizer, encoder

def evaluate_distilroberta_on_bootstrap_sample(model, tokenizer, texts, sentiment_labels, emotion_labels, 
                                        model_sentiment_encoder, model_emotion_encoder, 
                                        data_sentiment_encoder, data_emotion_encoder, 
                                        model_type="multitask", max_length=128):
    model.eval()
    
    if model_type == "multitask":
        sentiment_predictions = []
        emotion_predictions = []
        
        with torch.no_grad():
            for i in range(0, len(texts), 8):
                batch_texts = texts[i:i+8]
                
                inputs = tokenizer(
                    batch_texts,
                    return_tensors="pt",
                    truncation=True,
                    padding="max_length",
                    max_length=max_length
                )
                
                filtered_inputs = {
                    'input_ids': inputs['input_ids'].to(device),
                    'attention_mask': inputs['attention_mask'].to(device)
                }
                
                outputs = model(**filtered_inputs)
                
                sentiment_preds = torch.argmax(outputs['sentiment_logits'], dim=-1)
                emotion_preds = torch.argmax(outputs['emotion_logits'], dim=-1)
                
                for j in range(len(batch_texts)):
                    sent_id = sentiment_preds[j].item()
                    emot_id = emotion_preds[j].item()
                    
                    if sent_id >= len(model_sentiment_encoder.classes_):
                        sent_id = 0
                    if emot_id >= len(model_emotion_encoder.classes_):
                        emot_id = 0
                    
                    sentiment_predictions.append(sent_id)
                    emotion_predictions.append(emot_id)
        
        # Map predictions to data label space
        mapped_sentiment_preds = []
        mapped_emotion_preds = []
        
        for sent_pred, emot_pred in zip(sentiment_predictions, emotion_predictions):
            sent_class = model_sentiment_encoder.classes_[sent_pred]
            emot_class = model_emotion_encoder.classes_[emot_pred]
            
            try:
                mapped_sent = data_sentiment_encoder.transform([sent_class])[0]
                mapped_emot = data_emotion_encoder.transform([emot_class])[0]
            except ValueError:
                mapped_sent = 0
                mapped_emot = 0
            
            mapped_sentiment_preds.append(mapped_sent)
            mapped_emotion_preds.append(mapped_emot)
        
        # Calculate metrics
        sentiment_accuracy = accuracy_score(sentiment_labels, mapped_sentiment_preds)
        sentiment_f1 = f1_score(sentiment_labels, mapped_sentiment_preds, average='macro', zero_division=0)
        
        emotion_accuracy = accuracy_score(emotion_labels, mapped_emotion_preds)
        emotion_f1 = f1_score(emotion_labels, mapped_emotion_preds, average='macro', zero_division=0)
        
        return {
            'sentiment_accuracy': sentiment_accuracy,
            'sentiment_f1': sentiment_f1,
            'emotion_accuracy': emotion_accuracy,
            'emotion_f1': emotion_f1
        }
    
    else:
        # Single task evaluation logic here
        pass

def bootstrap_evaluation_distilroberta(model, tokenizer, data, model_sentiment_encoder, model_emotion_encoder,
                                data_sentiment_encoder, data_emotion_encoder, 
                                n_iterations=1000, sample_size=95):
    print(f"🔄 Starting distilroberta bootstrap evaluation...")
    print(f"   Iterations: {n_iterations}")
    print(f"   Sample size: {sample_size}")
    
    results = {
        'sentiment_accuracy': [],
        'sentiment_f1': [],
        'emotion_accuracy': [],
        'emotion_f1': []
    }
    
    texts = data['texts']
    sentiment_labels = data['sentiment_labels']
    emotion_labels = data['emotion_labels']
    n_samples = len(texts)
    
    for i in tqdm(range(n_iterations), desc="Bootstrap iterations"):
        # Bootstrap sample with replacement
        indices = np.random.choice(n_samples, size=sample_size, replace=True)
        
        sample_texts = [texts[idx] for idx in indices]
        sample_sentiment_labels = [sentiment_labels[idx] for idx in indices]
        sample_emotion_labels = [emotion_labels[idx] for idx in indices]
        
        # Evaluate on bootstrap sample
        metrics = evaluate_distilroberta_on_bootstrap_sample(
            model, tokenizer, sample_texts, sample_sentiment_labels, sample_emotion_labels,
            model_sentiment_encoder, model_emotion_encoder,
            data_sentiment_encoder, data_emotion_encoder
        )
        
        # Store results
        results['sentiment_accuracy'].append(metrics['sentiment_accuracy'])
        results['sentiment_f1'].append(metrics['sentiment_f1'])
        results['emotion_accuracy'].append(metrics['emotion_accuracy'])
        results['emotion_f1'].append(metrics['emotion_f1'])
    
    return results

print("distilroberta bootstrap analysis functions defined!")

distilroberta bootstrap analysis functions defined!


In [11]:
def run_distilroberta_bootstrap_analysis():
    print("🚀 Running distilroberta Bootstrap Analysis on General Datasets")
    print("=" * 60)
    
    # 1. Load external datasets
    print("\n📂 Loading general evaluation datasets...")
    sentiment_data, emotion_data = load_external_datasets()
    
    # 2. Prepare evaluation data
    print("Preparing SST-2 evaluation data...")
    sst2_eval_data = prepare_sst2_evaluation_data(sentiment_data)
    print("Preparing GoEmotions evaluation data...")
    goemotions_eval_data = prepare_goemotions_evaluation_data(emotion_data)
    print("Preparing multitask evaluation data...")
    multitask_eval_data = prepare_multitask_evaluation_data(sst2_eval_data, goemotions_eval_data)
    
    # 3. Load models
    # Load single task models
    sentiment_model_path = "./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_42"
    sentiment_model, sentiment_tokenizer, sentiment_encoder = load_distilroberta_model_for_bootstrap(
        sentiment_model_path, "sentiment"
    )
    
    emotion_model_path = "./distilroberta_trained_models_seeds/distilroberta_emotion_seed_42"
    emotion_model, emotion_tokenizer, emotion_encoder = load_distilroberta_model_for_bootstrap(
        emotion_model_path, "emotion"
    )
    
    # Load multitask model
    multitask_model_path = "./distilroberta_trained_models_seeds/distilroberta_multitask_seed_42"
    multitask_model, multitask_tokenizer, multitask_sent_encoder, multitask_emot_encoder = load_distilroberta_model_for_bootstrap(
        multitask_model_path, "multitask"
    )
    
    # 4. Run bootstrap evaluation
    print("\n🔄 Starting bootstrap evaluation...")
    n_iterations = 1000
    sample_size = 95
    
    # Initialize results dictionary for F1 scores
    f1_results = {
        'sentiment_single': [],
        'emotion_single': [],
        'multitask_sentiment': [],
        'multitask_emotion': []
    }
    
    # Run bootstrap iterations
    for i in tqdm(range(n_iterations), desc="Bootstrap iterations"):
        # Sample indices with replacement for each dataset
        sst2_indices = np.random.choice(len(sst2_eval_data['texts']), size=sample_size, replace=True)
        goemotions_indices = np.random.choice(len(goemotions_eval_data['texts']), size=sample_size, replace=True)
        
        # Prepare bootstrap samples
        sst2_sample = {
            'texts': [sst2_eval_data['texts'][i] for i in sst2_indices],
            'sentiment_labels': [sst2_eval_data['sentiment_labels'][i] for i in sst2_indices]
        }
        
        goemotions_sample = {
            'texts': [goemotions_eval_data['texts'][i] for i in goemotions_indices],
            'emotion_labels': [goemotions_eval_data['emotion_labels'][i] for i in goemotions_indices]
        }
        
        multitask_sample = {
            'texts': sst2_sample['texts'],  # Use SST-2 texts for multitask
            'sentiment_labels': sst2_sample['sentiment_labels'],
            'emotion_labels': [goemotions_eval_data['emotion_labels'][i] for i in sst2_indices]
        }
        
        # Evaluate single task models
        sentiment_results = evaluate_distilroberta_single_task(
            sentiment_model, sentiment_tokenizer, sentiment_encoder, 
            sst2_sample, 'sentiment'
        )
        f1_results['sentiment_single'].append(sentiment_results['macro_f1'])
        
        emotion_results = evaluate_distilroberta_single_task(
            emotion_model, emotion_tokenizer, emotion_encoder, 
            goemotions_sample, 'emotion'
        )
        f1_results['emotion_single'].append(emotion_results['macro_f1'])
        
        # Evaluate multitask model
        multitask_results = evaluate_distilroberta_multitask(
            multitask_model, multitask_tokenizer, 
            multitask_sent_encoder, multitask_emot_encoder,
            multitask_sample
        )
        f1_results['multitask_sentiment'].append(multitask_results['sentiment']['macro_f1'])
        f1_results['multitask_emotion'].append(multitask_results['emotion']['macro_f1'])
    
    # 5. Calculate and display statistics
    print("\ndistilroberta Bootstrap Analysis Results (General Datasets)")
    print("=" * 60)
    
    for model_name, f1_scores in f1_results.items():
        values = np.array(f1_scores)
        mean = np.mean(values)
        std = np.std(values)
        ci_lower = np.percentile(values, 2.5)
        ci_upper = np.percentile(values, 97.5)
        
        print(f"\n🎯 {model_name.replace('_', ' ').upper()} - F1")
        print(f"   Mean: {mean:.4f}")
        print(f"   Std:  {std:.4f}")
        print(f"   95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
    
    # 6. Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"./distilroberta_seed_analysis_results/distilroberta_bootstrap_general_datasets_{timestamp}.json"
    
    results_to_save = {
        model_name: {
            'values': [float(x) for x in values],
            'mean': float(np.mean(values)),
            'std': float(np.std(values)),
            'ci_lower': float(np.percentile(values, 2.5)),
            'ci_upper': float(np.percentile(values, 97.5))
        }
        for model_name, values in f1_results.items()
    }
    
    with open(results_file, 'w') as f:
        json.dump(results_to_save, f, indent=2)
    
    print(f"\n💾 Bootstrap results saved to: {results_file}")
    
    return results_to_save

In [12]:
# Cell 12: Run distilroberta Bootstrap Analysis

# Run bootstrap analysis
bootstrap_stats = run_distilroberta_bootstrap_analysis()

🚀 Running distilroberta Bootstrap Analysis on General Datasets

📂 Loading general evaluation datasets...
Loading external datasets...
SST-2 dataset loaded: 67349 train, 872 validation samples
GoEmotions dataset loaded: 43410 train, 5426 validation, 5427 test samples
Preparing SST-2 evaluation data...
Preparing SST-2 evaluation data...
✅ SST-2 evaluation data prepared: 872 samples
   Sentiment classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
Preparing GoEmotions evaluation data...
Preparing GoEmotions evaluation data...
✅ GoEmotions evaluation data prepared: 1000 samples
   Emotion classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]
Preparing multitask evaluation data...
Preparing multitask evaluation data...
✅ Multitask evaluation data prepared: 872 samples
📥 Loading distilroberta sentiment model from ./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_42...
📥 Loading distilrob

Bootstrap iterations: 100%|██████████| 1000/1000 [09:20<00:00,  1.78it/s]


distilroberta Bootstrap Analysis Results (General Datasets)

🎯 SENTIMENT SINGLE - F1
   Mean: 0.5723
   Std:  0.0230
   95% CI: [0.5205, 0.6121]

🎯 EMOTION SINGLE - F1
   Mean: 0.6889
   Std:  0.0527
   95% CI: [0.5782, 0.7901]

🎯 MULTITASK SENTIMENT - F1
   Mean: 0.5535
   Std:  0.0250
   95% CI: [0.5017, 0.5998]

🎯 MULTITASK EMOTION - F1
   Mean: 0.1114
   Std:  0.0181
   95% CI: [0.0766, 0.1471]

💾 Bootstrap results saved to: ./distilroberta_seed_analysis_results/distilroberta_bootstrap_general_datasets_20250725_173938.json


# Reddit specific dataset

In [30]:
# Cell 1: Setup and Imports for distilroberta Seed & Bootstrap Analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset, Dataset as HFDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
import json
import warnings
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import joblib
import random
from collections import Counter
import gc
from tqdm import tqdm
from scipy import stats

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create output directories
os.makedirs("./distilroberta_seed_analysis_results", exist_ok=True)
os.makedirs("./distilroberta_trained_models_seeds", exist_ok=True)

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print("✅ Libraries imported and setup complete!")

Using device: cuda
✅ Libraries imported and setup complete!


In [31]:
# Cell 2: Utility Functions for Analysis
def set_random_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def print_memory_usage():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory - Allocated: {allocated:.2f} GB, Cached: {cached:.2f} GB")

print("Utility functions defined")

Utility functions defined


In [32]:
# Cell 3: distilroberta Model Architectures
class DistilrobertaSingleTaskTransformer(nn.Module):
    
    def __init__(
        self,
        model_name: str = "distilroberta-base",
        num_classes: int = 3,
        hidden_dropout_prob: float = 0.1,
        attention_dropout_prob: float = 0.1,
        classifier_dropout: float = 0.1
    ):
        super().__init__()
        self.num_classes = num_classes
        
        # Load distilroberta model
        config = AutoConfig.from_pretrained(model_name)
        config.hidden_dropout_prob = hidden_dropout_prob
        config.attention_probs_dropout_prob = attention_dropout_prob
        
        self.distilroberta = AutoModel.from_pretrained(model_name, config=config)
        
        # Classification head
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(self.distilroberta.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        # Get distilroberta outputs
        outputs = self.distilroberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token
        pooled_output = self.dropout(pooled_output)
        
        # Classification
        logits = self.classifier(pooled_output)
        
        return {'logits': logits}

class DistilrobertaMultiTaskTransformer(nn.Module):
    
    def __init__(
        self,
        model_name: str = "distilroberta-base",
        sentiment_num_classes: int = 3,
        emotion_num_classes: int = 6,
        hidden_dropout_prob: float = 0.1,
        attention_dropout_prob: float = 0.1,
        classifier_dropout: float = 0.1
    ):
        super().__init__()
        self.sentiment_num_classes = sentiment_num_classes
        self.emotion_num_classes = emotion_num_classes
        
        # Load distilroberta model
        config = AutoConfig.from_pretrained(model_name)
        config.hidden_dropout_prob = hidden_dropout_prob
        config.attention_probs_dropout_prob = attention_dropout_prob
        
        self.distilroberta = AutoModel.from_pretrained(model_name, config=config)
        
        hidden_size = self.distilroberta.config.hidden_size
        
        # Task-specific attention layers
        self.sentiment_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        self.emotion_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        # Shared attention for common features
        self.shared_attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=8,
            dropout=attention_dropout_prob,
            batch_first=True
        )
        
        # Layer normalization
        self.sentiment_norm = nn.LayerNorm(hidden_size)
        self.emotion_norm = nn.LayerNorm(hidden_size)
        self.shared_norm = nn.LayerNorm(hidden_size)
        
        # Dropout layers
        self.sentiment_dropout = nn.Dropout(classifier_dropout)
        self.emotion_dropout = nn.Dropout(classifier_dropout)
        self.shared_dropout = nn.Dropout(classifier_dropout)
        
        # Classification heads
        self.sentiment_classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(hidden_size, sentiment_num_classes)
        )
        
        self.emotion_classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(hidden_size, emotion_num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for module in [self.sentiment_classifier, self.emotion_classifier]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.zeros_(layer.bias)
    
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Dict[str, torch.Tensor]:
        # Shared encoder
        encoder_outputs = self.distilroberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        sequence_output = encoder_outputs.last_hidden_state
        
        # Apply shared attention
        shared_attended, _ = self.shared_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        shared_attended = self.shared_norm(shared_attended + sequence_output)
        shared_attended = self.shared_dropout(shared_attended)
        shared_pooled = shared_attended[:, 0, :]
        
        outputs = {}
        
        # Sentiment branch
        sentiment_attended, _ = self.sentiment_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        sentiment_attended = self.sentiment_norm(sentiment_attended + sequence_output)
        sentiment_attended = self.sentiment_dropout(sentiment_attended)
        sentiment_pooled = sentiment_attended[:, 0, :]
        sentiment_features = torch.cat([shared_pooled, sentiment_pooled], dim=-1)
        sentiment_logits = self.sentiment_classifier(sentiment_features)
        outputs["sentiment_logits"] = sentiment_logits
        
        # Emotion branch
        emotion_attended, _ = self.emotion_attention(
            sequence_output, sequence_output, sequence_output,
            key_padding_mask=~attention_mask.bool()
        )
        emotion_attended = self.emotion_norm(emotion_attended + sequence_output)
        emotion_attended = self.emotion_dropout(emotion_attended)
        emotion_pooled = emotion_attended[:, 0, :]
        emotion_features = torch.cat([shared_pooled, emotion_pooled], dim=-1)
        emotion_logits = self.emotion_classifier(emotion_features)
        outputs["emotion_logits"] = emotion_logits
        
        return outputs

print("distilroberta model architectures defined")

distilroberta model architectures defined


In [33]:
# Cell 4: Dataset Classes for distilroberta
class DistilrobertaDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class DistilrobertaMultiTaskDataset(Dataset):
    def __init__(self, texts: List[str], sentiment_labels: List[int], 
                 emotion_labels: List[int], tokenizer, max_length: int = 128):
        self.texts = texts
        self.sentiment_labels = sentiment_labels
        self.emotion_labels = emotion_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        sentiment_label = self.sentiment_labels[idx]
        emotion_label = self.emotion_labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sentiment_labels': torch.tensor(sentiment_label, dtype=torch.long),
            'emotion_labels': torch.tensor(emotion_label, dtype=torch.long)
        }

print("distilroberta dataset classes defined")

distilroberta dataset classes defined


In [34]:
# Cell 5: Data Loading Functions for distilroberta Analysis
def load_external_datasets() -> Tuple[Dict, Dict]:
    print("Loading external datasets...")
    
    # Load SST-2 for sentiment
    try:
        sst2_dataset = load_dataset("sst2")
        sentiment_data = {
            'train': sst2_dataset['train'],
            'validation': sst2_dataset['validation']
        }
        print(f"✅ SST-2 dataset loaded: {len(sentiment_data['train'])} train samples")
    except Exception as e:
        print(f"❌ Could not load SST-2: {e}")
        raise
    
    # Load GoEmotions for emotion
    try:
        emotions_dataset = load_dataset("go_emotions", "simplified")
        emotion_data = {
            'train': emotions_dataset['train'],
            'validation': emotions_dataset['validation']
        }
        print(f"✅ GoEmotions dataset loaded: {len(emotion_data['train'])} train samples")
    except Exception as e:
        print(f"❌ Could not load GoEmotions: {e}")
        raise
    
    return sentiment_data, emotion_data

def prepare_reddit_evaluation_data(reddit_data_path: str) -> Dict:
    print(f"Loading Reddit evaluation data from {reddit_data_path}...")
    
    df = pd.read_csv(reddit_data_path)
    
    # Create label encoders that match distilroberta models
    sentiment_encoder = LabelEncoder()
    emotion_encoder = LabelEncoder()
    
    # Fit encoders
    sentiment_encoder.fit(df['sentiment'].tolist())
    emotion_encoder.fit(df['emotion'].tolist())
    
    reddit_data = {
        'texts': df['text_content'].tolist(),
        'sentiment_labels_text': df['sentiment'].tolist(),
        'emotion_labels_text': df['emotion'].tolist(),
        'sentiment_labels': sentiment_encoder.transform(df['sentiment'].tolist()),
        'emotion_labels': emotion_encoder.transform(df['emotion'].tolist()),
        'sentiment_encoder': sentiment_encoder,
        'emotion_encoder': emotion_encoder
    }
    
    print(f"✅ Reddit data prepared: {len(reddit_data['texts'])} samples")
    print(f"   Sentiment classes: {list(sentiment_encoder.classes_)}")
    print(f"   Emotion classes: {list(emotion_encoder.classes_)}")
    
    return reddit_data

def prepare_distilroberta_training_data(sentiment_data: Dict, emotion_data: Dict, max_samples: int = 5000):
    
    # Process sentiment data (SST-2 to 3 classes)
    sentiment_texts = sentiment_data['train']['sentence'][:max_samples]
    sentiment_labels_raw = sentiment_data['train']['label'][:max_samples]
    
    # Convert SST-2 binary to 3-class sentiment
    sentiment_labels = []
    for label in sentiment_labels_raw:
        if label == 0:  # Negative
            sentiment_labels.append(0)
        elif label == 1:  # Positive
            if np.random.random() < 0.15:  # 15% chance to be neutral
                sentiment_labels.append(1)  # Neutral
            else:
                sentiment_labels.append(2)  # Positive
    
    # Ensure we have all 3 classes
    if 1 not in sentiment_labels:
        neutral_indices = np.random.choice(len(sentiment_labels), size=100, replace=False)
        for idx in neutral_indices:
            sentiment_labels[idx] = 1
    
    # Process emotion data (filter to first 6 classes)
    emotion_texts_all = emotion_data['train']['text']
    emotion_labels_all = emotion_data['train']['labels']
    
    emotion_texts = []
    emotion_labels = []
    count = 0
    for i, label in enumerate(emotion_labels_all):
        if count >= max_samples:
            break
        if isinstance(label, list):
            if label and label[0] in range(6):
                emotion_texts.append(emotion_texts_all[i])
                emotion_labels.append(label[0])
                count += 1
        else:
            if label in range(6):
                emotion_texts.append(emotion_texts_all[i])
                emotion_labels.append(label)
                count += 1
    
    # Create encoders
    sentiment_encoder = LabelEncoder()
    emotion_encoder = LabelEncoder()
    sentiment_encoder.classes_ = np.array(['Negative', 'Neutral', 'Positive'])
    emotion_encoder.classes_ = np.array(['Anger', 'Fear', 'Joy', 'No Emotion', 'Sadness', 'Surprise'])
    
    return {
        'sentiment_data': {
            'texts': sentiment_texts,
            'labels': sentiment_labels,
            'encoder': sentiment_encoder
        },
        'emotion_data': {
            'texts': emotion_texts,
            'labels': emotion_labels,
            'encoder': emotion_encoder
        }
    }

print("Data loading functions defined")

Data loading functions defined


In [35]:
# Cell 6: distilroberta Training Functions with Best Parameters
def train_distilroberta_single_task(
    task_type: str,  # 'sentiment' or 'emotion'
    best_params: Dict,
    seed: int,
    training_data: Dict,
    max_samples: int = 5000
) -> Tuple[any, LabelEncoder]:
    
    print(f"🚀 Training distilroberta {task_type} model with seed {seed}")
    set_random_seed(seed)
    clear_memory()
    
    # Get appropriate data
    if task_type == 'sentiment':
        texts = training_data['sentiment_data']['texts'][:max_samples]
        labels = training_data['sentiment_data']['labels'][:max_samples]
        encoder = training_data['sentiment_data']['encoder']
        num_classes = 3
    else:  # emotion
        texts = training_data['emotion_data']['texts'][:max_samples]
        labels = training_data['emotion_data']['labels'][:max_samples]
        encoder = training_data['emotion_data']['encoder']
        num_classes = 6
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model
    model = DistilrobertaSingleTaskTransformer(
        model_name='distilroberta-base',
        num_classes=num_classes,
        hidden_dropout_prob=best_params['hidden_dropout_prob'],
        attention_dropout_prob=best_params['hidden_dropout_prob'],
        classifier_dropout=best_params['classifier_dropout']
    ).to(device)
    
    # Create dataset and dataloader
    dataset = DistilrobertaDataset(texts, labels, tokenizer, max_length=128)
    dataloader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=True)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
    
    total_steps = len(dataloader) * 3  # 3 epochs
    warmup_steps = int(total_steps * best_params['warmup_ratio'])
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    model.train()
    print(f"Starting training for 3 epochs...")
    
    for epoch in range(3):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_batch = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs['logits'], labels_batch)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/3, Average Loss: {avg_loss:.4f}")
    
    # Save model
    output_dir = f"./distilroberta_trained_models_seeds/distilroberta_{task_type}_seed_{seed}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model state dict
    torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
    
    # Save config
    config = {
        "model_name": "distilroberta-base",
        "num_classes": num_classes,
        "task_type": task_type,
        "model_type": "DistilrobertaSingleTaskTransformer"
    }
    with open(os.path.join(output_dir, "config.json"), 'w') as f:
        json.dump(config, f, indent=2)
    
    # Save tokenizer and encoder
    tokenizer.save_pretrained(output_dir)
    joblib.dump(encoder, os.path.join(output_dir, f'{task_type}_encoder.pkl'))
    
    print(f"✅ distilroberta {task_type} model trained and saved with seed {seed}")
    clear_memory()
    
    return model, encoder

def train_distilroberta_multitask(
    best_params: Dict,
    seed: int,
    training_data: Dict,
    max_samples: int = 2000
) -> Tuple[any, LabelEncoder, LabelEncoder]:
    
    print(f"🚀 Training distilroberta multitask model with seed {seed}")
    set_random_seed(seed)
    clear_memory()
    
    # Prepare multitask data (combine sentiment and emotion data)
    min_length = min(len(training_data['sentiment_data']['texts']), 
                     len(training_data['emotion_data']['texts']))
    min_length = min(min_length, max_samples)
    
    combined_texts = training_data['sentiment_data']['texts'][:min_length]
    combined_sentiment_labels = training_data['sentiment_data']['labels'][:min_length]
    combined_emotion_labels = training_data['emotion_data']['labels'][:min_length]
    
    sentiment_encoder = training_data['sentiment_data']['encoder']
    emotion_encoder = training_data['emotion_data']['encoder']
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model
    model = DistilrobertaMultiTaskTransformer(
        model_name='distilroberta-base',
        sentiment_num_classes=3,
        emotion_num_classes=6,
        hidden_dropout_prob=best_params['hidden_dropout_prob'],
        attention_dropout_prob=best_params['hidden_dropout_prob'],
        classifier_dropout=best_params['classifier_dropout']
    ).to(device)
    
    # Create dataset and dataloader
    dataset = DistilrobertaMultiTaskDataset(
        combined_texts, combined_sentiment_labels, combined_emotion_labels, 
        tokenizer, max_length=128
    )
    dataloader = DataLoader(dataset, batch_size=best_params['batch_size'], shuffle=True)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
    
    total_steps = len(dataloader) * 3  # 3 epochs
    warmup_steps = int(total_steps * best_params['warmup_ratio'])
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # Loss functions
    sentiment_criterion = nn.CrossEntropyLoss()
    emotion_criterion = nn.CrossEntropyLoss()
    
    alpha = best_params['alpha']
    
    # Training loop
    model.train()
    print(f"Starting training for 3 epochs...")
    
    for epoch in range(3):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            sentiment_labels = batch['sentiment_labels'].to(device)
            emotion_labels = batch['emotion_labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Calculate losses
            sentiment_loss = sentiment_criterion(outputs['sentiment_logits'], sentiment_labels)
            emotion_loss = emotion_criterion(outputs['emotion_logits'], emotion_labels)
            
            # Combined loss
            total_loss_batch = alpha * sentiment_loss + (1 - alpha) * emotion_loss
            total_loss += total_loss_batch.item()
            
            # Backward pass
            total_loss_batch.backward()
            optimizer.step()
            scheduler.step()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/3, Average Loss: {avg_loss:.4f}")
    
    # Save model
    output_dir = f"./distilroberta_trained_models_seeds/distilroberta_multitask_seed_{seed}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model state dict
    torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
    
    # Save config
    config = {
        "model_name": "distilroberta-base",
        "sentiment_num_classes": 3,
        "emotion_num_classes": 6,
        "model_type": "DistilrobertaMultiTaskTransformer"
    }
    with open(os.path.join(output_dir, "config.json"), 'w') as f:
        json.dump(config, f, indent=2)
    
    # Save tokenizer and encoders
    tokenizer.save_pretrained(output_dir)
    joblib.dump(sentiment_encoder, os.path.join(output_dir, 'sentiment_encoder.pkl'))
    joblib.dump(emotion_encoder, os.path.join(output_dir, 'emotion_encoder.pkl'))
    
    print(f"distilroberta multitask model trained and saved with seed {seed}")
    clear_memory()
    
    return model, sentiment_encoder, emotion_encoder

print("distilroberta training functions defined")

distilroberta training functions defined


In [36]:
# Cell 7: Evaluation Functions for distilroberta Models
def evaluate_distilroberta_single_task(model, tokenizer, label_encoder, reddit_data: Dict, task_type: str) -> Dict:
    
    model.eval()
    model.to(device)
    
    texts = reddit_data['texts']
    true_labels = reddit_data[f'{task_type}_labels']
    
    predictions = []
    confidences = []
    
    with torch.no_grad():
        for i in range(0, len(texts), 16):  # Batch size 16
            batch_texts = texts[i:i+16]
            
            # Tokenize
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=128
            )
            
            inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
            
            # Forward pass
            outputs = model(**inputs)
            logits = outputs['logits']
            probs = F.softmax(logits, dim=-1)
            preds = torch.argmax(logits, dim=-1)
            
            # Collect results
            for j in range(len(batch_texts)):
                pred_id = preds[j].item()
                confidence = probs[j][pred_id].item()
                
                # Handle out of range predictions
                if pred_id >= len(label_encoder.classes_):
                    pred_id = 0
                
                predictions.append(pred_id)
                confidences.append(confidence)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    macro_f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'predictions': predictions,
        'confidences': confidences,
        'true_labels': true_labels
    }

def evaluate_distilroberta_multitask(model, tokenizer, sentiment_encoder, emotion_encoder, 
                               reddit_data: Dict, max_length: int = 128) -> Dict:
    
    model.eval()
    model.to(device)
    
    texts = reddit_data['texts']
    true_sentiment_labels = reddit_data['sentiment_labels']
    true_emotion_labels = reddit_data['emotion_labels']
    
    sentiment_predictions = []
    emotion_predictions = []
    sentiment_confidences = []
    emotion_confidences = []
    
    with torch.no_grad():
        for i in range(0, len(texts), 8):  # Smaller batch size for multitask
            batch_texts = texts[i:i+8]
            
            # Tokenize
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=max_length
            )
            
            inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
            
            # Forward pass
            outputs = model(**inputs)
            
            # Process sentiment
            sentiment_logits = outputs['sentiment_logits']
            sentiment_probs = F.softmax(sentiment_logits, dim=-1)
            sentiment_preds = torch.argmax(sentiment_logits, dim=-1)
            
            # Process emotion
            emotion_logits = outputs['emotion_logits']
            emotion_probs = F.softmax(emotion_logits, dim=-1)
            emotion_preds = torch.argmax(emotion_logits, dim=-1)
            
            # Collect results
            for j in range(len(batch_texts)):
                # Sentiment
                sent_id = sentiment_preds[j].item()
                sent_conf = sentiment_probs[j][sent_id].item()
                if sent_id >= len(sentiment_encoder.classes_):
                    sent_id = 0
                sentiment_predictions.append(sent_id)
                sentiment_confidences.append(sent_conf)
                
                # Emotion
                emot_id = emotion_preds[j].item()
                emot_conf = emotion_probs[j][emot_id].item()
                if emot_id >= len(emotion_encoder.classes_):
                    emot_id = 0
                emotion_predictions.append(emot_id)
                emotion_confidences.append(emot_conf)
    
    # Calculate metrics
    sentiment_accuracy = accuracy_score(true_sentiment_labels, sentiment_predictions)
    sentiment_f1 = f1_score(true_sentiment_labels, sentiment_predictions, average='macro', zero_division=0)
    
    emotion_accuracy = accuracy_score(true_emotion_labels, emotion_predictions)
    emotion_f1 = f1_score(true_emotion_labels, emotion_predictions, average='macro', zero_division=0)
    
    return {
        'sentiment': {
            'accuracy': sentiment_accuracy,
            'macro_f1': sentiment_f1,
            'predictions': sentiment_predictions,
            'confidences': sentiment_confidences
        },
        'emotion': {
            'accuracy': emotion_accuracy,
            'macro_f1': emotion_f1,
            'predictions': emotion_predictions,
            'confidences': emotion_confidences
        },
        'combined_accuracy': (sentiment_accuracy + emotion_accuracy) / 2,
        'combined_f1': (sentiment_f1 + emotion_f1) / 2
    }

print("distilroberta evaluation functions defined")

distilroberta evaluation functions defined


In [37]:
# Cell 8: distilroberta Random Seed Analysis Function
def run_distilroberta_seed_analysis(
    reddit_data_path: str = "annotated_reddit_posts.csv",
    seeds: List[int] = [42, 123, 456, 789, 999],
    max_training_samples: int = 3000
):
    
    print("🎲 STARTING distilroberta RANDOM SEED ANALYSIS")
    print("=" * 70)
    print(f"Seeds to test: {seeds}")
    print(f"Max training samples per dataset: {max_training_samples}")
    
    # Load external datasets
    print("\n📂 Loading external datasets...")
    sentiment_data, emotion_data = load_external_datasets()
    
    # Prepare training data
    print("\n🔄 Preparing distilroberta training data...")
    training_data = prepare_distilroberta_training_data(sentiment_data, emotion_data, max_training_samples)
    
    # Load Reddit evaluation data
    print("\n📂 Loading Reddit evaluation data...")
    reddit_data = prepare_reddit_evaluation_data(reddit_data_path)
    
    # Define best parameters for each distilroberta model
    best_params = {
        'sentiment': {
            'learning_rate': 3.65445235521325e-05,
            'batch_size': 16,
            'warmup_ratio': 0.15986584841970367,
            'weight_decay': 0.02404167763981929,
            'hidden_dropout_prob': 0.13119890406724052,
            'classifier_dropout': 0.1116167224336399
        },
        'emotion': {
            'learning_rate': 3.65445235521325e-05, 
            'batch_size': 16,
            'warmup_ratio': 0.15986584841970367,
            'weight_decay': 0.02404167763981929,
            'hidden_dropout_prob': 0.13119890406724052,
            'classifier_dropout': 0.1116167224336399
        },
        'multitask': {
            'learning_rate': 4.166863122305896e-05,
            'batch_size': 16,
            'warmup_ratio': 0.15142344384136117,
            'weight_decay': 0.06331731119758383,
            'hidden_dropout_prob': 0.10929008254399955,
            'classifier_dropout': 0.22150897038028766,
            'alpha': 0.4341048247374583
        }
    }
  
    # Store results for each seed
    all_results = {}
    
    for seed in seeds:
        print(f"\n🌱 TRAINING AND EVALUATING distilroberta WITH SEED {seed}")
        print("-" * 60)
        
        seed_results = {}
        
        # 1. Train and evaluate distilroberta Sentiment
        print(f"\n1️⃣ distilroberta Sentiment (Seed {seed})")
        model, encoder = train_distilroberta_single_task(
            'sentiment', best_params['sentiment'], seed, 
            training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_{seed}")
        
        # Evaluate
        results = evaluate_distilroberta_single_task(model, tokenizer, encoder, reddit_data, 'sentiment')
        seed_results['distilroberta_sentiment'] = results
        print(f"   Accuracy: {results['accuracy']:.4f}, Macro F1: {results['macro_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        # 2. Train and evaluate distilroberta Emotion
        print(f"\n2️⃣ distilroberta Emotion (Seed {seed})")
        model, encoder = train_distilroberta_single_task(
            'emotion', best_params['emotion'], seed,
            training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_emotion_seed_{seed}")
        
        # Evaluate
        results = evaluate_distilroberta_single_task(model, tokenizer, encoder, reddit_data, 'emotion')
        seed_results['distilroberta_emotion'] = results
        print(f"   Accuracy: {results['accuracy']:.4f}, Macro F1: {results['macro_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        # 3. Train and evaluate distilroberta Multitask
        print(f"\n3️⃣ distilroberta Multitask (Seed {seed})")
        model, sent_enc, emot_enc = train_distilroberta_multitask(
            best_params['multitask'], seed, training_data, max_training_samples
        )
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(f"./distilroberta_trained_models_seeds/distilroberta_multitask_seed_{seed}")
        
        # Evaluate
        results = evaluate_distilroberta_multitask(
            model, tokenizer, sent_enc, emot_enc, reddit_data, 128
        )
        seed_results['distilroberta_multitask'] = results
        print(f"   Sentiment - Accuracy: {results['sentiment']['accuracy']:.4f}, F1: {results['sentiment']['macro_f1']:.4f}")
        print(f"   Emotion - Accuracy: {results['emotion']['accuracy']:.4f}, F1: {results['emotion']['macro_f1']:.4f}")
        print(f"   Combined - Accuracy: {results['combined_accuracy']:.4f}, F1: {results['combined_f1']:.4f}")
        
        del model, tokenizer
        clear_memory()
        
        all_results[seed] = seed_results
        
        print(f"\n✅ Completed evaluation for seed {seed}")
    
    # Analyze stability across seeds
    print(f"\n📊 ANALYZING distilroberta STABILITY ACROSS SEEDS")
    print("=" * 70)
    
    stability_analysis = analyze_distilroberta_seed_stability(all_results, seeds)
    
    # Save results
    save_distilroberta_results(all_results, stability_analysis, seeds)
    
    return all_results, stability_analysis

print("✅ distilroberta random seed analysis function defined!")

✅ distilroberta random seed analysis function defined!


In [38]:
# Cell 9: distilroberta Stability Analysis Functions
def analyze_distilroberta_seed_stability(all_results: Dict, seeds: List[int]) -> Dict:
    
    stability_stats = {}
    
    # Define model-task combinations
    evaluations = [
        ('distilroberta_sentiment', 'sentiment'),
        ('distilroberta_emotion', 'emotion'),
        ('distilroberta_multitask', 'sentiment'),
        ('distilroberta_multitask', 'emotion')
    ]
    
    for model_name, task in evaluations:
        print(f"\n🔍 {model_name.upper()} - {task.upper()}")
        
        accuracies = []
        f1_scores = []
        
        for seed in seeds:
            if model_name in all_results[seed]:
                result = all_results[seed][model_name]
                
                if model_name.endswith('_multitask'):
                    acc = result[task]['accuracy']
                    f1 = result[task]['macro_f1']
                else:
                    acc = result['accuracy']
                    f1 = result['macro_f1']
                
                accuracies.append(acc)
                f1_scores.append(f1)
        
        if accuracies:
            acc_mean = np.mean(accuracies)
            acc_std = np.std(accuracies)
            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            
            stability_stats[f"{model_name}_{task}"] = {
                'accuracy_mean': acc_mean,
                'accuracy_std': acc_std,
                'f1_mean': f1_mean,
                'f1_std': f1_std,
                'accuracy_values': accuracies,
                'f1_values': f1_scores
            }
            
            print(f"   Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
            print(f"   Macro F1: {f1_mean:.4f} ± {f1_std:.4f}")
    
    return stability_stats

def save_distilroberta_results(all_results: Dict, stability_analysis: Dict, seeds: List[int]):
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save raw results
    results_file = f"./distilroberta_seed_analysis_results/distilroberta_raw_results_{timestamp}.json"
    with open(results_file, 'w') as f:
        # Convert numpy types to Python types for JSON serialization
        serializable_results = {}
        for seed, seed_results in all_results.items():
            serializable_results[str(seed)] = {}
            for model, results in seed_results.items():
                if isinstance(results, dict):
                    serializable_results[str(seed)][model] = {}
                    for key, value in results.items():
                        if isinstance(value, dict):
                            serializable_results[str(seed)][model][key] = {
                                k: float(v) if isinstance(v, (np.floating, np.integer)) else 
                                   [float(x) if isinstance(x, (np.floating, np.integer)) else x for x in v] if isinstance(v, list) else v
                                for k, v in value.items()
                            }
                        else:
                            serializable_results[str(seed)][model][key] = float(value) if isinstance(value, (np.floating, np.integer)) else value
        
        json.dump(serializable_results, f, indent=2)
    
    # Save stability analysis
    stability_file = f"./distilroberta_seed_analysis_results/distilroberta_stability_analysis_{timestamp}.json"
    with open(stability_file, 'w') as f:
        serializable_stability = {}
        for key, stats in stability_analysis.items():
            serializable_stability[key] = {
                k: float(v) if isinstance(v, (np.floating, np.integer)) else 
                   [float(x) for x in v] if isinstance(v, list) else v
                for k, v in stats.items()
            }
        json.dump(serializable_stability, f, indent=2)
    
    # Create summary report
    summary_file = f"./distilroberta_seed_analysis_results/distilroberta_summary_report_{timestamp}.txt"
    with open(summary_file, 'w') as f:
        f.write("distilroberta RANDOM SEED ANALYSIS SUMMARY REPORT\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Seeds tested: {seeds}\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("STABILITY ANALYSIS (Mean ± Std)\n")
        f.write("-" * 40 + "\n")
        
        for key, stats in stability_analysis.items():
            model_task = key.replace('_', ' ').title()
            f.write(f"\n{model_task}:\n")
            f.write(f"  Accuracy: {stats['accuracy_mean']:.4f} ± {stats['accuracy_std']:.4f}\n")
            f.write(f"  Macro F1: {stats['f1_mean']:.4f} ± {stats['f1_std']:.4f}\n")
        
        f.write(f"\nBest Performers (by mean F1 score):\n")
        f.write("-" * 30 + "\n")
        
        # Find best performers
        sentiment_best = max([k for k in stability_analysis.keys() if 'sentiment' in k], 
                           key=lambda x: stability_analysis[x]['f1_mean'])
        emotion_best = max([k for k in stability_analysis.keys() if 'emotion' in k], 
                         key=lambda x: stability_analysis[x]['f1_mean'])
        
        f.write(f"Sentiment: {sentiment_best.replace('_', ' ').title()} ")
        f.write(f"(F1: {stability_analysis[sentiment_best]['f1_mean']:.4f})\n")
        f.write(f"Emotion: {emotion_best.replace('_', ' ').title()} ")
        f.write(f"(F1: {stability_analysis[emotion_best]['f1_mean']:.4f})\n")
    
    print(f"\n💾 distilroberta results saved:")
    print(f"   Raw results: {results_file}")
    print(f"   Stability analysis: {stability_file}")
    print(f"   Summary report: {summary_file}")

print("distilroberta stability analysis functions defined")

distilroberta stability analysis functions defined


In [39]:
# Cell 10: Run distilroberta Random Seed Analysis (Fixed)

# Clear any previous results
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Run distilroberta random seed analysis with the fixed saving function
try:
    all_results, stability_analysis = run_distilroberta_seed_analysis(
        reddit_data_path="annotated_reddit_posts.csv",
        seeds=[42, 123, 456, 789, 999],  # 5 different seeds
        max_training_samples=3000  # Reduced for faster training
    )
    
    print("\n🎉 DISTILROBERTA RANDOM SEED ANALYSIS COMPLETED!")
    print("=" * 60)
    print("Check the './distilroberta_seed_analysis_results/' directory for detailed results.")
    
    # Display quick summary
    print("\n📊 QUICK STABILITY SUMMARY:")
    print("-" * 40)
    
    for model_name in ['DISTILROBERTA_SENTIMENT', 'DISTILROBERTA_EMOTION', 'DISTILROBERTA_MULTITASK']:
        if model_name in stability_analysis:
            print(f"\n{model_name}:")
            for task in ['sentiment', 'emotion']:
                if task in stability_analysis[model_name]:
                    metrics = stability_analysis[model_name][task]
                    print(f"  {task.title()}:")
                    print(f"    Accuracy: {metrics.get('accuracy_mean', 0):.3f} ± {metrics.get('accuracy_std', 0):.3f}")
                    print(f"    F1 Score: {metrics.get('f1_mean', 0):.3f} ± {metrics.get('f1_std', 0):.3f}")
                    print(f"    Stability: {metrics.get('stability_score', 0):.3f}")

except Exception as e:
    print(f"❌ Error during analysis: {str(e)}")
    print("🔧 Try restarting the kernel and running cells 1-9 again.")

🎲 STARTING distilroberta RANDOM SEED ANALYSIS
Seeds to test: [42, 123, 456, 789, 999]
Max training samples per dataset: 3000

📂 Loading external datasets...
Loading external datasets...
✅ SST-2 dataset loaded: 67349 train samples
✅ GoEmotions dataset loaded: 43410 train samples

🔄 Preparing distilroberta training data...

📂 Loading Reddit evaluation data...
Loading Reddit evaluation data from annotated_reddit_posts.csv...
✅ Reddit data prepared: 95 samples
   Sentiment classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
   Emotion classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]

🌱 TRAINING AND EVALUATING distilroberta WITH SEED 42
------------------------------------------------------------

1️⃣ distilroberta Sentiment (Seed 42)
🚀 Training distilroberta sentiment model with seed 42
Starting training for 3 epochs...
Epoch 1/3, Average Loss: 0.7703
Epoch 2/3, Average Loss: 0.5131
Epoch 3/3, A

In [13]:
# Cell 11: distilroberta Bootstrap Analysis Functions
def load_distilroberta_model_for_bootstrap(model_path: str, model_type: str):
    print(f"📥 Loading distilroberta {model_type} model from {model_path}...")
    
    # Load config
    with open(os.path.join(model_path, 'config.json'), 'r') as f:
        config = json.load(f)
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    if model_type == "multitask":
        # Load multitask model
        model = DistilrobertaMultiTaskTransformer(
            model_name="distilroberta-base",
            sentiment_num_classes=config['sentiment_num_classes'],
            emotion_num_classes=config['emotion_num_classes']
        )
        
        # Load weights
        state_dict = torch.load(os.path.join(model_path, 'pytorch_model.bin'), map_location=device)
        model.load_state_dict(state_dict)
        model.to(device)
        
        # Load encoders
        sentiment_encoder = joblib.load(os.path.join(model_path, 'sentiment_encoder.pkl'))
        emotion_encoder = joblib.load(os.path.join(model_path, 'emotion_encoder.pkl'))
        
        return model, tokenizer, sentiment_encoder, emotion_encoder
        
    else:
        # Load single-task model
        model = DistilrobertaSingleTaskTransformer(
            model_name="distilroberta-base",
            num_classes=config['num_classes']
        )
        
        # Load weights
        state_dict = torch.load(os.path.join(model_path, 'pytorch_model.bin'), map_location=device)
        model.load_state_dict(state_dict)
        model.to(device)
        
        # Load encoder
        encoder = joblib.load(os.path.join(model_path, f'{config["task_type"]}_encoder.pkl'))
        
        return model, tokenizer, encoder

def evaluate_distilroberta_on_bootstrap_sample(model, tokenizer, texts, sentiment_labels, emotion_labels, 
                                        model_sentiment_encoder, model_emotion_encoder, 
                                        data_sentiment_encoder, data_emotion_encoder, 
                                        model_type="multitask", max_length=128):
    model.eval()
    
    if model_type == "multitask":
        sentiment_predictions = []
        emotion_predictions = []
        
        with torch.no_grad():
            for i in range(0, len(texts), 8):
                batch_texts = texts[i:i+8]
                
                inputs = tokenizer(
                    batch_texts,
                    return_tensors="pt",
                    truncation=True,
                    padding="max_length",
                    max_length=max_length
                )
                
                filtered_inputs = {
                    'input_ids': inputs['input_ids'].to(device),
                    'attention_mask': inputs['attention_mask'].to(device)
                }
                
                outputs = model(**filtered_inputs)
                
                sentiment_preds = torch.argmax(outputs['sentiment_logits'], dim=-1)
                emotion_preds = torch.argmax(outputs['emotion_logits'], dim=-1)
                
                for j in range(len(batch_texts)):
                    sent_id = sentiment_preds[j].item()
                    emot_id = emotion_preds[j].item()
                    
                    if sent_id >= len(model_sentiment_encoder.classes_):
                        sent_id = 0
                    if emot_id >= len(model_emotion_encoder.classes_):
                        emot_id = 0
                    
                    sentiment_predictions.append(sent_id)
                    emotion_predictions.append(emot_id)
        
        # Map predictions to data label space
        mapped_sentiment_preds = []
        mapped_emotion_preds = []
        
        for sent_pred, emot_pred in zip(sentiment_predictions, emotion_predictions):
            sent_class = model_sentiment_encoder.classes_[sent_pred]
            emot_class = model_emotion_encoder.classes_[emot_pred]
            
            try:
                mapped_sent = data_sentiment_encoder.transform([sent_class])[0]
                mapped_emot = data_emotion_encoder.transform([emot_class])[0]
            except ValueError:
                mapped_sent = 0
                mapped_emot = 0
            
            mapped_sentiment_preds.append(mapped_sent)
            mapped_emotion_preds.append(mapped_emot)
        
        # Calculate metrics
        sentiment_accuracy = accuracy_score(sentiment_labels, mapped_sentiment_preds)
        sentiment_f1 = f1_score(sentiment_labels, mapped_sentiment_preds, average='macro', zero_division=0)
        
        emotion_accuracy = accuracy_score(emotion_labels, mapped_emotion_preds)
        emotion_f1 = f1_score(emotion_labels, mapped_emotion_preds, average='macro', zero_division=0)
        
        return {
            'sentiment_accuracy': sentiment_accuracy,
            'sentiment_f1': sentiment_f1,
            'emotion_accuracy': emotion_accuracy,
            'emotion_f1': emotion_f1
        }
    
    else:
        # Single task evaluation logic here
        pass

def bootstrap_evaluation_distilroberta(model, tokenizer, data, model_sentiment_encoder, model_emotion_encoder,
                                data_sentiment_encoder, data_emotion_encoder, 
                                n_iterations=1000, sample_size=95):
    print(f"🔄 Starting distilroberta bootstrap evaluation...")
    print(f"   Iterations: {n_iterations}")
    print(f"   Sample size: {sample_size}")
    
    results = {
        'sentiment_accuracy': [],
        'sentiment_f1': [],
        'emotion_accuracy': [],
        'emotion_f1': []
    }
    
    texts = data['texts']
    sentiment_labels = data['sentiment_labels']
    emotion_labels = data['emotion_labels']
    n_samples = len(texts)
    
    for i in tqdm(range(n_iterations), desc="Bootstrap iterations"):
        # Bootstrap sample with replacement
        indices = np.random.choice(n_samples, size=sample_size, replace=True)
        
        sample_texts = [texts[idx] for idx in indices]
        sample_sentiment_labels = [sentiment_labels[idx] for idx in indices]
        sample_emotion_labels = [emotion_labels[idx] for idx in indices]
        
        # Evaluate on bootstrap sample
        metrics = evaluate_distilroberta_on_bootstrap_sample(
            model, tokenizer, sample_texts, sample_sentiment_labels, sample_emotion_labels,
            model_sentiment_encoder, model_emotion_encoder,
            data_sentiment_encoder, data_emotion_encoder
        )
        
        # Store results
        results['sentiment_accuracy'].append(metrics['sentiment_accuracy'])
        results['sentiment_f1'].append(metrics['sentiment_f1'])
        results['emotion_accuracy'].append(metrics['emotion_accuracy'])
        results['emotion_f1'].append(metrics['emotion_f1'])
    
    return results

print("distilroberta bootstrap analysis functions defined!")

distilroberta bootstrap analysis functions defined!


In [16]:
def run_distilroberta_bootstrap_analysis():
    print("🚀 Running distilroberta Bootstrap Analysis on General Datasets")
    print("=" * 60)
    
    # 1. Load all models (single task and multitask)
    print("\n📥 Loading models...")
    
    # Load single task models
    sentiment_model_path = "./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_42"
    sentiment_model, sentiment_tokenizer, sentiment_encoder = load_distilroberta_model_for_bootstrap(
        sentiment_model_path, "sentiment"
    )
    
    emotion_model_path = "./distilroberta_trained_models_seeds/distilroberta_emotion_seed_42"
    emotion_model, emotion_tokenizer, emotion_encoder = load_distilroberta_model_for_bootstrap(
        emotion_model_path, "emotion"
    )
    
    # Load multitask model
    multitask_model_path = "./distilroberta_trained_models_seeds/distilroberta_multitask_seed_42"
    multitask_model, multitask_tokenizer, multitask_sent_encoder, multitask_emot_encoder = load_distilroberta_model_for_bootstrap(
        multitask_model_path, "multitask"
    )
    
    # 2. Load evaluation data
    print("\n📂 Loading evaluation datasets...")
    reddit_data = prepare_reddit_evaluation_data("annotated_reddit_posts.csv")
    
    # 3. Run bootstrap evaluation for each model
    print("\n🔄 Starting bootstrap evaluation...")
    n_iterations = 1000
    sample_size = 95
    
    # Initialize results dictionary for F1 scores
    f1_results = {
        'sentiment_single': [],
        'emotion_single': [],
        'multitask_sentiment': [],
        'multitask_emotion': []
    }
    
    # Run bootstrap iterations
    for i in tqdm(range(n_iterations), desc="Bootstrap iterations"):
        # Sample indices with replacement
        indices = np.random.choice(len(reddit_data['texts']), size=sample_size, replace=True)
        
        # Prepare bootstrap sample
        sample_data = {
            'texts': [reddit_data['texts'][i] for i in indices],
            'sentiment_labels': [reddit_data['sentiment_labels'][i] for i in indices],
            'emotion_labels': [reddit_data['emotion_labels'][i] for i in indices]
        }
        
        # Evaluate single task models
        sentiment_results = evaluate_distilroberta_single_task(
            sentiment_model, sentiment_tokenizer, sentiment_encoder, 
            sample_data, 'sentiment'
        )
        f1_results['sentiment_single'].append(sentiment_results['macro_f1'])
        
        emotion_results = evaluate_distilroberta_single_task(
            emotion_model, emotion_tokenizer, emotion_encoder, 
            sample_data, 'emotion'
        )
        f1_results['emotion_single'].append(emotion_results['macro_f1'])
        
        # Evaluate multitask model
        multitask_results = evaluate_distilroberta_multitask(
            multitask_model, multitask_tokenizer, 
            multitask_sent_encoder, multitask_emot_encoder,
            sample_data
        )
        f1_results['multitask_sentiment'].append(multitask_results['sentiment']['macro_f1'])
        f1_results['multitask_emotion'].append(multitask_results['emotion']['macro_f1'])
    
    # 4. Calculate and display statistics
    print("\ndistilroberta Bootstrap Analysis Results")
    print("=" * 60)
    
    for model_name, f1_scores in f1_results.items():
        values = np.array(f1_scores)
        mean = np.mean(values)
        std = np.std(values)
        ci_lower = np.percentile(values, 2.5)
        ci_upper = np.percentile(values, 97.5)
        
        print(f"\n🎯 {model_name.replace('_', ' ').upper()} - F1")
        print(f"   Mean: {mean:.4f}")
        print(f"   Std:  {std:.4f}")
        print(f"   95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
    
    # 5. Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"./distilroberta_seed_analysis_results/distilroberta_bootstrap_results_{timestamp}.json"
    
    results_to_save = {
        model_name: {
            'values': [float(x) for x in values],
            'mean': float(np.mean(values)),
            'std': float(np.std(values)),
            'ci_lower': float(np.percentile(values, 2.5)),
            'ci_upper': float(np.percentile(values, 97.5))
        }
        for model_name, values in f1_results.items()
    }
    
    with open(results_file, 'w') as f:
        json.dump(results_to_save, f, indent=2)
    
    print(f"\n💾 Bootstrap results saved to: {results_file}")
    
    return results_to_save

def prepare_reddit_evaluation_data(reddit_data_path: str) -> Dict:
    print(f"Loading Reddit evaluation data from {reddit_data_path}...")
    
    df = pd.read_csv(reddit_data_path)
    
    # Create label encoders that match distilroberta models
    sentiment_encoder = LabelEncoder()
    emotion_encoder = LabelEncoder()
    
    # Fit encoders
    sentiment_encoder.fit(df['sentiment'].tolist())
    emotion_encoder.fit(df['emotion'].tolist())
    
    reddit_data = {
        'texts': df['text_content'].tolist(),
        'sentiment_labels_text': df['sentiment'].tolist(),
        'emotion_labels_text': df['emotion'].tolist(),
        'sentiment_labels': sentiment_encoder.transform(df['sentiment'].tolist()),
        'emotion_labels': emotion_encoder.transform(df['emotion'].tolist()),
        'sentiment_encoder': sentiment_encoder,
        'emotion_encoder': emotion_encoder
    }
    
    print(f"✅ Reddit data prepared: {len(reddit_data['texts'])} samples")
    print(f"   Sentiment classes: {list(sentiment_encoder.classes_)}")
    print(f"   Emotion classes: {list(emotion_encoder.classes_)}")
    
    return reddit_data


In [17]:
# Cell 12: Run distilroberta Bootstrap Analysis

# Run bootstrap analysis
bootstrap_stats = run_distilroberta_bootstrap_analysis()

🚀 Running distilroberta Bootstrap Analysis on General Datasets

📥 Loading models...
📥 Loading distilroberta sentiment model from ./distilroberta_trained_models_seeds/distilroberta_sentiment_seed_42...
📥 Loading distilroberta emotion model from ./distilroberta_trained_models_seeds/distilroberta_emotion_seed_42...
📥 Loading distilroberta multitask model from ./distilroberta_trained_models_seeds/distilroberta_multitask_seed_42...

📂 Loading evaluation datasets...
Loading Reddit evaluation data from annotated_reddit_posts.csv...
✅ Reddit data prepared: 95 samples
   Sentiment classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
   Emotion classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]

🔄 Starting bootstrap evaluation...


Bootstrap iterations: 100%|██████████| 1000/1000 [10:01<00:00,  1.66it/s]


distilroberta Bootstrap Analysis Results

🎯 SENTIMENT SINGLE - F1
   Mean: 0.4016
   Std:  0.0418
   95% CI: [0.3165, 0.4781]

🎯 EMOTION SINGLE - F1
   Mean: 0.0763
   Std:  0.0196
   95% CI: [0.0398, 0.1184]

🎯 MULTITASK SENTIMENT - F1
   Mean: 0.3053
   Std:  0.0397
   95% CI: [0.2315, 0.3828]

🎯 MULTITASK EMOTION - F1
   Mean: 0.1272
   Std:  0.0192
   95% CI: [0.0897, 0.1658]

💾 Bootstrap results saved to: ./distilroberta_seed_analysis_results/distilroberta_bootstrap_results_20250725_190934.json
